In [9]:
import pandas as pd
import os


relative_data_path = '../../data/raw/' 
file_name = 'X07_성폭력 영향요소 융합데이터.csv' # ⚠️ 컴퓨터에서 변경한 가장 단순한 이름으로 가정
file_path = os.path.join(relative_data_path, file_name)

# 2. CSV 파일 읽기
try:
    df_influence = pd.read_csv(file_path, encoding='utf-8')
    
    # 데이터가 잘 불러와졌는지 확인하기 위해 상위 5개 행을 출력
    print("데이터 불러오기 성공!")
    print(df_influence.head())
    
except FileNotFoundError:
    print(f"오류: '{file_path}' 파일을 찾을 수 없습니다. 파일 경로와 이름을 다시 확인해주세요.")
except Exception as e:
    # 한글 인코딩 문제로 오류가 날 경우를 대비한 추가 처리
    print(f"다른 인코딩으로 재시도 중... 현재 오류: {e}")
    try:
        df_influence = pd.read_csv(file_path, encoding='cp949') # 윈도우 기본 인코딩
        print("데이터 불러오기 성공 (cp949)")
        print(df_influence.head())
    except:
        print("파일 불러오기에 실패했습니다. 파일이 깨졌거나 인코딩 문제가 심각합니다.")

데이터 불러오기 성공!
   OCRN_YR PLCST_NM  SVLNC  DTVC  STKNG  RPRT_CASCNT_WHOL  RPRT_SEX_MALE  \
0  ﻿"2022"       제천     36    90     27               153             40   
1     2022     청주상당    106   231    132               469            116   
2     2022       옥천      9     8      7                24              8   
3     2022       영동     15    16     10                41             16   
4     2022       충주     61   226     59               346             75   

   RPRT_SEX_FMLE  RPRT_SEX_UNKWN  RPRT_SEX_ETC  ...  OCRN_TIZN_16  \
0             87              26             0  ...             5   
1            327              26             0  ...            18   
2             16               0             0  ...             2   
3             25               0             0  ...             1   
4            232              39             0  ...            10   

   OCRN_TIZN_17  OCRN_TIZN_18  OCRN_TIZN_19  OCRN_TIZN_20  OCRN_TIZN_21  \
0             4             4           

In [10]:
address_column_name = 'PLCST_NM' 

    # --- 3. 전국구 데이터에서 '서울' 포함 행만 필터링 ---
    # 문자열 처리 함수(str)와 contains()를 사용하여 '서울'이라는 문자열을 포함하는 행만 선택
    # isnull()과 | (OR 연산자)를 사용하여 혹시 모를 결측치도 함께 포함합니다.
df_violence = df_influence[
    df_influence[address_column_name].str.contains('서울', na=False)].copy()

print("✅ 서울 지역 데이터 필터링 완료!")
print(f"원본 데이터 크기: {len(df_influence)} 행")
print(f"서울 데이터 크기: {len(df_seoul)} 행")
print("\n[서울 필터링된 데이터 미리보기]")
print(df_seoul.head())

✅ 서울 지역 데이터 필터링 완료!
원본 데이터 크기: 242 행
서울 데이터 크기: 31 행

[서울 필터링된 데이터 미리보기]
   OCRN_YR PLCST_NM  SVLNC  DTVC  STKNG  RPRT_CASCNT_WHOL  RPRT_SEX_MALE  \
19    2022     서울용산    274   347    171               792            249   
20    2022     서울성동    151   254    165               570            152   
21    2022     서울중부    190   197    100               487            174   
22    2022     서울광진    408   668    242              1318            428   
23    2022    서울동대문    236   477    213               926            258   

    RPRT_SEX_FMLE  RPRT_SEX_UNKWN  RPRT_SEX_ETC  ...  OCRN_TIZN_16  \
19            465              78             0  ...            19   
20            366              52             0  ...            18   
21            258              55             0  ...            22   
22            748             142             0  ...            45   
23            576              92             0  ...            29   

    OCRN_TIZN_17  OCRN_TIZN_18  OCRN_TIZN_19  OCR

In [11]:
df_violence['PLCST_NM']

19     서울용산
20     서울성동
21     서울중부
22     서울광진
23    서울동대문
24     서울중랑
25     서울성북
26     서울종암
27     서울강북
29     서울도봉
30     서울노원
31     서울서부
32     서울은평
33    서울서대문
34     서울마포
35     서울양천
36     서울관악
37     서울서초
38     서울방배
39     서울강남
40     서울수서
41     서울송파
42     서울강동
47     서울강서
48     서울구로
49     서울금천
50    서울영등포
51     서울동작
62     서울종로
63     서울혜화
64    서울남대문
Name: PLCST_NM, dtype: object

In [12]:
# df_violence가 성폭력 데이터프레임이라고 가정

# 1. '여성 잠재적 위험 지수'를 계산할 세 가지 핵심 컬럼 지정
risk_columns = ['SVLNC', 'DTVC', 'STKNG']

# 2. 이 세 컬럼의 값을 행(경찰서) 단위로 모두 합산 (axis=1)
# 이 값이 바로 '경찰서별 여성 잠재적 위험 지수'가 돼!
df_violence['여성_잠재적_위험_지수'] = df_violence[risk_columns].sum(axis=1)

print("✅ '여성_잠재적_위험_지수' 컬럼 생성 완료!")
print(df_violence[['PLCST_NM', '여성_잠재적_위험_지수']].head())

✅ '여성_잠재적_위험_지수' 컬럼 생성 완료!
   PLCST_NM  여성_잠재적_위험_지수
19     서울용산           792
20     서울성동           570
21     서울중부           487
22     서울광진          1318
23    서울동대문           926


총 신고 건수 대비 긴급 신고 건수의 비율이 높을수록 사건 발생시 위험도가 매우 높다 > 경찰의 신속 출동 인프라 보강이 시급하다.

In [13]:
# df_violence가 성폭력 데이터프레임이라고 가정
import numpy as np

# 1. '긴급 신고 비율' 계산 (총 신고 건수가 0일 경우 오류 방지를 위해 np.where 사용)
# 공식: (긴급 접수 건수 / 총 신고 건수) * 100
df_violence['긴급_신고_비율(%)'] = np.where(
    df_violence['RPRT_CASCNT_WHOL'] == 0, # 총 신고 건수가 0이면
    0, # 비율도 0
    (df_violence['ACPT_EMERG_TYPE_EMERG'] / df_violence['RPRT_CASCNT_WHOL']) * 100
).round(2)

print("✅ '긴급_신고_비율(%)' 컬럼 생성 완료!")
print(df_violence[['PLCST_NM', 'RPRT_CASCNT_WHOL', 'ACPT_EMERG_TYPE_EMERG', '긴급_신고_비율(%)']].head())

✅ '긴급_신고_비율(%)' 컬럼 생성 완료!
   PLCST_NM  RPRT_CASCNT_WHOL  ACPT_EMERG_TYPE_EMERG  긴급_신고_비율(%)
19     서울용산               792                     91        11.49
20     서울성동               570                     62        10.88
21     서울중부               487                     62        12.73
22     서울광진              1318                    159        12.06
23    서울동대문               926                    116        12.53


In [14]:
# df_violence가 성폭력 데이터프레임이라고 가정

# 1. 심야 시간대 컬럼 리스트 지정 (22시, 23시, 00시, 01시, 02시, 03시, 04시)
midnight_zones = [
    'OCRN_TIZN_22', 'OCRN_TIZN_23', # 저녁 10시, 11시
    'OCRN_TIZN_00', 'OCRN_TIZN_01', 'OCRN_TIZN_02', 'OCRN_TIZN_03', 'OCRN_TIZN_04' # 새벽 0시 ~ 4시
]

# 2. 심야 시간대 발생 건수를 모두 합산
df_violence['심야_발생_총합'] = df_violence[midnight_zones].sum(axis=1)

# 3. 전체 발생 건수 합산 (24시간 전체)
# 'OCRN_TIZN_00'부터 'OCRN_TIZN_23'까지 모든 시간대를 합쳐야 함
all_time_zones = [f'OCRN_TIZN_{str(i).zfill(2)}' for i in range(24)]
df_violence['전체_시간대_총합'] = df_violence[all_time_zones].sum(axis=1)

# 4. '심야 취약 지수' (전체 대비 심야 비율) 계산
df_violence['심야_취약_비율(%)'] = np.where(
    df_violence['전체_시간대_총합'] == 0, # 전체 발생이 0이면
    0, # 비율도 0
    (df_violence['심야_발생_총합'] / df_violence['전체_시간대_총합']) * 100
).round(2)

print("✅ '심야_취약_비율(%)' 컬럼 생성 완료!")
print(df_violence[['PLCST_NM', '심야_발생_총합', '전체_시간대_총합', '심야_취약_비율(%)']].head())

✅ '심야_취약_비율(%)' 컬럼 생성 완료!
   PLCST_NM  심야_발생_총합  전체_시간대_총합  심야_취약_비율(%)
19     서울용산       402        792        50.76
20     서울성동       258        570        45.26
21     서울중부       211        487        43.33
22     서울광진       672       1318        50.99
23    서울동대문       389        926        42.01


In [15]:
# df_violence가 성폭력 데이터프레임이라고 가정
import numpy as np

# --- 1. 퇴근 후 (Evening Zone) ---
evening_zones = [
    'OCRN_TIZN_18', 'OCRN_TIZN_19', 'OCRN_TIZN_20', 'OCRN_TIZN_21' # 저녁 6시 ~ 9시
]
df_violence['퇴근후_발생_총합'] = df_violence[evening_zones].sum(axis=1)

df_violence['퇴근후_취약_비율(%)'] = np.where(
    df_violence['전체_시간대_총합'] == 0, # 전체 발생이 0이면
    0, 
    (df_violence['퇴근후_발생_총합'] / df_violence['전체_시간대_총합']) * 100
).round(2)


# --- 2. 주간 (Day Zone) ---
day_zones = [
    'OCRN_TIZN_09', 'OCRN_TIZN_10', 'OCRN_TIZN_11', 'OCRN_TIZN_12', 
    'OCRN_TIZN_13', 'OCRN_TIZN_14', 'OCRN_TIZN_15', 'OCRN_TIZN_16', 'OCRN_TIZN_17' # 오전 9시 ~ 오후 5시
]
df_violence['주간_발생_총합'] = df_violence[day_zones].sum(axis=1)

df_violence['주간_취약_비율(%)'] = np.where(
    df_violence['전체_시간대_총합'] == 0, # 전체 발생이 0이면
    0, 
    (df_violence['주간_발생_총합'] / df_violence['전체_시간대_총합']) * 100
).round(2)

print("✅ '퇴근후' 및 '주간' 취약 비율 컬럼 생성 완료!")
print(df_violence[['PLCST_NM', '심야_취약_비율(%)', '퇴근후_취약_비율(%)', '주간_취약_비율(%)']].head())

✅ '퇴근후' 및 '주간' 취약 비율 컬럼 생성 완료!
   PLCST_NM  심야_취약_비율(%)  퇴근후_취약_비율(%)  주간_취약_비율(%)
19     서울용산        50.76         12.75        24.75
20     서울성동        45.26         16.49        25.79
21     서울중부        43.33         18.07        28.34
22     서울광진        50.99         16.77        22.23
23    서울동대문        42.01         20.19        27.43
